In [1]:
import torch
torch.manual_seed(0)

from chop.ir.graph.mase_graph import MaseGraph

from chop.passes.graph.analysis import (
    init_metadata_analysis_pass,
    add_common_metadata_analysis_pass,
    add_hardware_metadata_analysis_pass,
    report_node_type_analysis_pass,
)

from chop.passes.graph.transforms import (
    emit_verilog_top_transform_pass,
    emit_internal_rtl_transform_pass,
    emit_bram_transform_pass,
    emit_cocotb_transform_pass,
    quantize_transform_pass,
)

from chop.tools.logger import set_logging_verbosity

set_logging_verbosity("debug")

import toml
import torch
import torch.nn as nn
import os
# os.environ["PATH"] = "/vol/bitbucket/oa321/verilator/verilator/bin:" + os.environ["PATH"]
os.environ["MODULE"] = "top"
!verilator

/home/omar/Documents/year-4/adls/mase/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to debug


Usage:
        verilator --help
        verilator --version
        verilator --binary -j 0 [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --cc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --sc [options] [source_files.v]... [opt_c_files.cpp/c/cc/a/o/so]
        verilator --lint-only -Wall [source_files.v]...



In [2]:
class MLP(torch.nn.Module):
    """
    Toy FC model for digit recognition on MNIST
    """

    def __init__(self) -> None:
        super().__init__()

        self.fc1 = nn.Linear(4, 8)

    def forward(self, x):
        x = torch.flatten(x, start_dim=1, end_dim=-1)
        x = torch.nn.functional.relu(self.fc1(x))
        return x

In [3]:
mlp = MLP()
mg = MaseGraph(model=mlp)

# Provide a dummy input for the graph so it can use for tracing
batch_size = 1
x = torch.randn((batch_size, 2, 2))
dummy_in = {"x": x}

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(
    mg, {"dummy_in": dummy_in, "add_value": False}
)

DEBUG    graph():
    %x : [num_users=1] = placeholder[target=x]
    %flatten : [num_users=1] = call_function[target=torch.flatten](args = (%x,), kwargs = {start_dim: 1, end_dim: -1})
    %fc1 : [num_users=1] = call_module[target=fc1](args = (%flatten,), kwargs = {})
    %relu : [num_users=1] = call_function[target=torch.nn.functional.relu](args = (%fc1,), kwargs = {inplace: False})
    return relu


In [4]:
config_file = os.path.join(
    os.path.abspath(""),
    "..",
    "..",
    "configs",
    "tests",
    "quantize",
    "fixed.toml",
)
with open(config_file, "r") as f:
    quan_args = toml.load(f)["passes"]["quantize"]
mg, _ = quantize_transform_pass(mg, quan_args)

_ = report_node_type_analysis_pass(mg)

# Update the metadata
for node in mg.fx_graph.nodes:
    for arg, arg_info in node.meta["mase"]["common"]["args"].items():
        if isinstance(arg_info, dict):
            arg_info["type"] = "fixed"
            arg_info["precision"] = [8, 3]
    for result, result_info in node.meta["mase"]["common"]["results"].items():
        if isinstance(result_info, dict):
            result_info["type"] = "fixed"
            result_info["precision"] = [8, 3]

INFO     Inspecting graph [add_common_node_type_analysis_pass]
INFO     
Node name    Fx Node op     Mase type            Mase op      Value type
-----------  -------------  -------------------  -----------  ------------
x            placeholder    placeholder          placeholder  NA
flatten      call_function  implicit_func        flatten      float
fc1          call_module    module_related_func  linear       fixed
relu         call_function  module_related_func  relu         fixed
output       output         output               output       NA


In [5]:
mg, _ = add_hardware_metadata_analysis_pass(mg)

In [6]:
from pathlib import  Path

mg, _ = emit_verilog_top_transform_pass(mg)
mg, _ = emit_internal_rtl_transform_pass(mg)

INFO     Emitting Verilog...
INFO     Emitting internal components...


In [7]:
mg, _ = emit_bram_transform_pass(mg)

INFO     Emitting BRAM...
DEBUG    Emitting DAT file for node: fc1, parameter: weight
DEBUG    ROM module weight successfully written into /home/omar/.mase/top/hardware/rtl/fc1_weight_source.sv
DEBUG    Init data weight successfully written into /home/omar/.mase/top/hardware/rtl/fc1_weight_rom.dat
DEBUG    Emitting DAT file for node: fc1, parameter: bias
DEBUG    ROM module bias successfully written into /home/omar/.mase/top/hardware/rtl/fc1_bias_source.sv
DEBUG    Init data bias successfully written into /home/omar/.mase/top/hardware/rtl/fc1_bias_rom.dat


In [8]:
mg, _ = emit_cocotb_transform_pass(mg)

INFO     Emitting testbench...


In [9]:
from chop.actions import simulate

simulate(skip_build=False, skip_test=False, waves=True)

INFO: Running command perl /usr/local/bin/verilator -cc --exe -Mdir /home/omar/Documents/year-4/adls/mase/tasks/tutorial_hw/sim_build -DCOCOTB_SIM=1 --top-module top --vpi --public-flat-rw --prefix Vtop -o top -LDFLAGS '-Wl,-rpath,/home/omar/Documents/year-4/adls/mase/venv/lib/python3.11/site-packages/cocotb/libs -L/home/omar/Documents/year-4/adls/mase/venv/lib/python3.11/site-packages/cocotb/libs -lcocotbvpi_verilator' --trace -Wno-fatal -Wno-lint -Wno-style --trace --trace-fst --trace-structs --trace-depth 3 -I/home/omar/.mase/top/hardware/rtl -I/home/omar/Documents/year-4/adls/mase/src/mase_components/activation_layers/rtl -I/home/omar/Documents/year-4/adls/mase/src/mase_components/normalization_layers/rtl -I/home/omar/Documents/year-4/adls/mase/src/mase_components/vision_models/rtl -I/home/omar/Documents/year-4/adls/mase/src/mase_components/scalar_operators/rtl -I/home/omar/Documents/year-4/adls/mase/src/mase_components/vivado/rtl -I/home/omar/Documents/year-4/adls/mase/src/mase_co

INFO     Build finished. Time taken: 21.95s


g++    verilator.o verilated.o verilated_dpi.o verilated_vpi.o verilated_fst_c.o verilated_threads.o Vtop__ALL.a   -Wl,-rpath,/home/omar/Documents/year-4/adls/mase/venv/lib/python3.11/site-packages/cocotb/libs -L/home/omar/Documents/year-4/adls/mase/venv/lib/python3.11/site-packages/cocotb/libs -lcocotbvpi_verilator -lz  -pthread -lpthread -latomic   -o top
rm Vtop__ALL.verilator_deplist.tmp
make: Leaving directory '/home/omar/Documents/year-4/adls/mase/tasks/tutorial_hw/sim_build'
INFO: Running command /home/omar/Documents/year-4/adls/mase/tasks/tutorial_hw/sim_build/top --trace in directory /home/omar/Documents/year-4/adls/mase/tasks/tutorial_hw/sim_build
     -.--ns INFO     gpi                                ..mbed/gpi_embed.cpp:108  in set_program_name_in_venv        Using Python virtual environment interpreter at /home/omar/Documents/year-4/adls/mase/venv/bin/python
     -.--ns INFO     gpi                                ../gpi/GpiCommon.cpp:101  in gpi_print_registered_impl     

/home/omar/Documents/year-4/adls/mase/src/mase_cocotb/driver.py:25: DeprecationWarning: This method is now private.
  self._thread = cocotb.scheduler.add(self._send_thread())
/home/omar/Documents/year-4/adls/mase/src/mase_cocotb/monitor.py:27: DeprecationWarning: This method is now private.
  self._thread = cocotb.scheduler.add(self._recv_thread())
/usr/lib/python3.11/tempfile.py:1073: ResourceWarning: Implicitly cleaning up <TemporaryDirectory '/tmp/tmp1u6482bl'>
  _warnings.warn(warn_message, ResourceWarning)


{'data_in_0': tensor([[0.4963, 0.7682, 0.0885, 0.1320]])}
    60.00ns DEBUG    cocotb.driver.StreamDriver         Sent [3, 6, 0, 1]
   260.00ns DEBUG    cocotb.monitor.StreamMonitor       Observed output beat [0, 0, 0, 0]
   260.00ns DEBUG    cocotb.monitor.StreamMonitor       Got [0, 0, 0, 0], Expected [0, 0, 0, 0]
   280.00ns DEBUG    cocotb.monitor.StreamMonitor       Observed output beat [0, 6, 1, 0]
   280.00ns DEBUG    cocotb.monitor.StreamMonitor       Got [0, 6, 1, 0], Expected [0, 6, 2, 0]
   280.00ns INFO     cocotb.monitor.StreamMonitor       Monitor has been drained.
   280.00ns INFO     cocotb.regression                  test passed
   280.00ns INFO     cocotb.regression                  **************************************************************************************
                                                        ** TEST                          STATUS  SIM TIME (ns)  REAL TIME (s)  RATIO (ns/s) **
                                                        ****

INFO     Test finished. Time taken: 9.23s


INFO: Results file: /home/omar/Documents/year-4/adls/mase/tasks/tutorial_hw/sim_build/results.xml
